<a href="https://colab.research.google.com/github/ariescacy9/trabajo_data_science_2/blob/main/proyecto_bloque2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MODELO PARA DETECCIÓN DE DERRUMBES

Los paises con una diversa topografía que incluye altas montañas y valles profundos, es frecuentemente golpeado por intensas precipitaciones, lo que lo hace altamente susceptible a deslizamientos de tierra inducidos por lluvias. Un deslizamiento de tierra es el movimiento de masas de roca, escombros o tierra cuesta abajo y puede resultar en una pérdida significativa de vidas y propiedades. Un inventario de deslizamientos de tierra de alta calidad es esencial no solo para el análisis de riesgos y peligros por deslizamientos, sino también para apoyar las decisiones de las agencias sobre la mitigación y prevención de estos peligros.

**Reto:**
Predecir zonas con posibilidad de derrumbe (1: Sí, 0: No)

##**Archivos**
*train_set.csv* - Datos de entrenamiento con columna target
*test_set.csv *- Datos de evaluación sin el target
*sample_submission.csv* - Archivo de ejemplo de como deberian realizar el submission
Cada muestra está compuesta por datos de 25 celdas, cubriendo un área de 625 m2. Cada celda representa un área de 5 x 5 m2 y tiene nueve características (como se ve en los cuadrantes mostrados luego). Para una muestra de deslizamiento de tierra, la celda 13 es la ubicación del deslizamiento, y las otras celdas son las áreas vecinas. Para una muestra sin deslizamiento de tierra, no hay un deslizamiento de tierra registrado dentro del área de la muestra.

Libraries

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle
from sklearn.preprocessing import PolynomialFeatures
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from google.colab import files

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Creacion de dataframes

In [4]:
# Cargar datos
train_data = pd.read_csv("/content/drive/MyDrive/base_de_datos/train_set.csv")
test_data = pd.read_csv("/content/drive/MyDrive/base_de_datos/test_set.csv")

# Seleccionar características y variable objetivo
X = train_data.drop(['ID', 'target'], axis=1)
y = train_data['target']

Analisis exploratorio

In [5]:
# Calcular la matriz de correlación, ya que podrían proporcionar información redundante.
correlation_matrix = X.corr()

# Crear una matriz triangular superior de la matriz de correlación
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Definir el umbral
threshold = 0.8

# Encontrar y eliminar características altamente correlacionadas
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]
X = X.drop(to_drop, axis=1)
X_test = pd.get_dummies(test_data.drop(columns=to_drop))
X_test = X_test.drop(['ID'], axis=1)

In [6]:
# Cantidad de caracteristicas dropeadas
upper_triangle
print(len(to_drop))

157


In [7]:
# Aplicar técnicas de aumento de datos
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_augmented = poly.fit_transform(X)
X_test_augmented = poly.fit_transform(X_test)
X_train_augmented, y_train_augmented = shuffle(X_train_augmented, y)

Model testing

In [8]:
# Splitear datos para el entrenamiento de prueba
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train_augmented, y_train_augmented, test_size=0.2, random_state=1)

In [9]:
# Aplicar oversampling o undersampling
oversampler = RandomOverSampler(sampling_strategy='minority')
X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)

# O bien, usar undersampling
# undersampler = RandomUnderSampler(sampling_strategy='majority')
# X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

Definir configuracion optima del modelo para los datos proporcionados

In [12]:
# Definir los hiperparámetros a ajustar
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
}

# Inicializar el clasificador
rf_model = RandomForestClassifier()

# Realizar búsqueda de cuadrícula
grid_search = GridSearchCV(rf_model,
                           param_grid,
                           scoring='recall',
                           cv=5)
# grid_search.fit(X_train, y_train)
grid_search.fit(X_resampled, y_resampled)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

Mejores hiperparámetros: {'max_depth': 20, 'n_estimators': 100}


Model training

In [13]:
model = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=1)
# model.fit(X_train, y_train)
model.fit(X_resampled, y_resampled)

RandomForestClassifier(max_depth=10, n_estimators=500, random_state=1)

Score Test

In [14]:
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy on validation set: {accuracy:.5f}")
precision = precision_score(y_val, y_pred)
print(f"Precision: {precision:.5f}")
recall = recall_score(y_val, y_pred)
print(f"Recall: {recall:.5f}")
print(f"final score: {2*precision*recall/(precision+recall):.5f}")
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))

Accuracy on validation set: 0.80671
Precision: 0.58705
Recall: 0.70699
final score: 0.64146
Confusion Matrix:
 [[964 185]
 [109 263]]
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.84      0.87      1149
           1       0.59      0.71      0.64       372

    accuracy                           0.81      1521
   macro avg       0.74      0.77      0.75      1521
weighted avg       0.82      0.81      0.81      1521



In [15]:
# predictions = model.predict(X_test)
predictions = model.predict(X_test_augmented)
output = pd.DataFrame({'ID': test_data.ID, 'target': predictions})
output.to_csv('submission.csv', index=False)
files.view("submission.csv")
print("Your submission was successfully saved!")

<IPython.core.display.Javascript object>

Your submission was successfully saved!


Export with Pickle

In [16]:
# Guardar el modelo
with open('proyecto_bloque2.pkl', 'wb') as file:
    pickle.dump(model, file)

# Luego, para cargar el modelo más tarde
with open('proyecto_bloque2.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [17]:
files.download('proyecto_bloque2.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>